# Python Program for Feedforward Models

## BoW

In [ ]:
"""
Title            : Python Program for Feedforward Models
Dataset          : CR (as an example) 
Feature Extration: Bag-of-Words
Author           : Diardano Raihan
"""

##=========================Import Libraries======================#
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from string import punctuation
from collections import Counter
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.text import Tokenizer

#==============================Step 1============================#
# Load the dataset
corpus = pd.read_pickle('../../0_data/CR/CR.pkl')

#==============================Step 2============================#
# Define functions needed for:
# 1. Cleaning the text and turning into tokens
# 2. Creating a vocabulary list for BoW

stopwords = stopwords.words('english')
stemmer = PorterStemmer()
    
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # filter out stop words
    tokens = [w for w in tokens if not w in stopwords]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) >= 1]
    # Stem the token
    tokens = [stemmer.stem(token) for token in tokens]
    return tokens

vocab = Counter()

def add_doc_to_vocab(docs, vocab):
    '''
    input:
        docs: a list of sentences (docs)
        vocab: a vocabulary dictionary
    output:
        return an updated vocabulary
    '''
    for doc in docs:
        tokens = clean_doc(doc)
        vocab.update(tokens)
    return vocab

def doc_to_line(doc):
    tokens = clean_doc(doc)
    # filter by vocab
    tokens = [token for token in tokens if token in vocab]
    line = ' '.join([token for token in tokens])
    return line

def clean_docs(docs):
    lines = []
    for doc in docs:
        line = doc_to_line(doc)
        lines.append(line)
    return lines

def create_tokenizer(sentence):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

#==============================Step 3============================#
# Define the feedforward model SNN-a
def train_mlp_1(train_x, train_y, batch_size = 50, epochs = 10, 
                verbose =2):
    
    n_words = train_x.shape[1]
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=50, 
                              activation='relu', 
                              input_shape=(n_words,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=1, activation='sigmoid')
    ])
    
    model.compile(loss = 'binary_crossentropy', 
                  optimizer = 'adam', 
                  metrics = ['accuracy'])
    model.fit(train_x, train_y, batch_size, epochs, verbose)
    return model

# Define the feedforward model SNN-b
def train_mlp_2(train_x, train_y, batch_size = 50, epochs = 10, 
                verbose =2):
    
    n_words = train_x.shape[1]
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense( units=100, activation='relu', 
                              input_shape=(n_words,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=1, activation='sigmoid')
    ])
    
    model.compile(loss = 'binary_crossentropy', 
                  optimizer = 'adam', 
                  metrics = ['accuracy'])
    model.fit(train_x, train_y, batch_size, epochs, verbose)
    return model

# Define the feedforward model SNN-b
def train_mlp_3(train_x, train_y, batch_size = 50, epochs = 10, 
                verbose =2):
    
    n_words = train_x.shape[1]
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense( units=100, activation='relu', 
                              input_shape=(n_words,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=50, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=1, activation='sigmoid')
    ])
    
    model.compile(loss = 'binary_crossentropy', 
                  optimizer = 'adam', 
                  metrics = ['accuracy'])
    model.fit(train_x, train_y, batch_size, epochs, verbose)
    return model

# Define the early stopping callbacks
callbacks = tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            min_delta=0, 
            patience=5, verbose=2, 
            mode='auto', 
            restore_best_weights=True)


#==============================Step 4============================#
# Begin the training process with BoW four word scoring options

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Run Experiment of 4 different modes of BoW word scoring
modes = ['binary', 'count', 'tfidf', 'freq']
results_1 = pd.DataFrame()
results_2 = pd.DataFrame()
results_3 = pd.DataFrame()


for mode in modes:
    print('mode: ', mode)
    acc_list_1 = []
    acc_list_2 = []
    acc_list_3 = []
    
    
    # kfold.split() will return set indices for each split
    for train, test in kfold.split(sentences):
        # Instantiate a vocab object
        vocab = Counter()

        train_x, test_x = [], []
        train_y, test_y = [], []

        for i in train:
            train_x.append(sentences[i])
            train_y.append(labels[i])

        for i in test:
            test_x.append(sentences[i])
            test_y.append(labels[i])

        # Turn the labels into a numpy array
        train_y = np.array(train_y)
        test_y = np.array(test_y)

        # Define a vocabulary for each fold
        vocab = add_doc_to_vocab(train_x, vocab)
        # print('The number of vocab: ', len(vocab))

        # Clean the sentences
        train_x = clean_docs(train_x, vocab)
        test_x = clean_docs(test_x, vocab)

        # encode data using freq mode
        Xtrain, Xtest = prepare_data(train_x, test_x, mode)

        # train the model
        model_1 = train_mlp_1(Xtrain, train_y, Xtest, test_y, 
                              verbose=1)
        model_2 = train_mlp_2(Xtrain, train_y, Xtest, test_y, 
                              verbose=1)
        model_3 = train_mlp_3(Xtrain, train_y, Xtest, test_y, 
                              verbose=1)

        # evaluate the model
        loss_1, acc_1 = model_1.evaluate(Xtest, test_y, verbose=0)
        loss_2, acc_2 = model_2.evaluate(Xtest, test_y, verbose=0)
        loss_3, acc_3 = model_3.evaluate(Xtest, test_y, verbose=0)

        acc_list_1.append(acc_1)
        acc_list_2.append(acc_2)
        acc_list_3.append(acc_3)
        
    results_1[mode] = acc_list_1
    results_2[mode] = acc_list_2
    results_3[mode] = acc_list_3

print(results_1)
print(results_2)
print(results_3)

#==============================Step 5============================#
# Save the dataframe into excel file
results_1.to_excel('BoW_SNN_a.xlsx', sheet_name='model_1')
results_2.to_excel('BoW_SNN_b.xlsx', sheet_name='model_2')
results_3.to_excel('BoW_SNN_c.xlsx', sheet_name='model_3')
#================================================================#

## WE

In [ ]:
"""
Title            : Python Program for Feedforward Models
Dataset          : CR (as an example) 
Feature Extration: Average Word2Vec
Author           : Diardano Raihan
"""

##=========================Import Libraries======================#
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from string import punctuation
from collections import Counter
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim.models import KeyedVectors
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.text import Tokenizer

#==============================Step 1============================#
# Load the dataset
corpus = pd.read_pickle('../../0_data/CR/CR.pkl')
# Load the Word2Vec
word2vec = KeyedVectors.load_word2vec_format(
            '../GoogleNews-vectors-negative300.bin', 
            binary=True)

#==============================Step 2============================#
# Define functions needed to:
# 1. Clean the text
# 2. Convert the text into vectors based on Word2Vec

def clean_doc(sentences, word_index):
    clean_sentences = []
    for sentence in sentences:
        sentence = sentence.lower().split()
        clean_word = []
        for word in sentence:
            if word in word_index:
                clean_word.append(word)
        clean_sentence = ' '.join(clean_word)
        clean_sentences.append(clean_sentence)
    return clean_sentences

def sentence_to_avg(sentence, word_to_vec_map):
    
    # Split sentence into list of lower case words
    words = (sentence.lower()).split()

    # Initialize the average word vector
    avg = np.zeros(word2vec.word_vec('i').shape)
    
    # Average the word vectors
    total = 0
    count = 0
    for w in words:
        if w in word_to_vec_map:
            total += word_to_vec_map.word_vec(w)
            count += 1
            
    if count!= 0:
        avg = total/count
    else:
        avg
    return avg

# Encode Sentence into Word2Vec Representation
def encoded_sentences(sentences):

    encoded_sentences = []

    for sentence in sentences:

        encoded_sentence = sentence_to_avg(sentence, word2vec)
        encoded_sentences.append(encoded_sentence)

    encoded_sentences = np.array(encoded_sentences)
    return encoded_sentences

#==============================Step 3============================#
# Model Definition

# Define the feedforward model SNN-a
def define_model(input_length=300):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=50, activation='relu', 
                              input_shape=(input_length,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=1, activation='sigmoid')
    ])
    
    model.compile(loss = 'binary_crossentropy', 
                  optimizer = 'adam', 
                  metrics = ['accuracy'])
    return model

# Define the feedforward model SNN-b
def define_model_2(input_length=300):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=100, 
                              activation='relu', 
                              input_shape=(input_length,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=1, activation='sigmoid')
    ])
    
    model.compile(loss = 'binary_crossentropy', 
                  optimizer = 'adam', 
                  metrics = ['accuracy'])
    return model

# Define the feedforward model SNN-c
def define_model_3(input_length=300):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=100, 
                              activation='relu', 
                              input_shape=(input_length,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=50, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=1, activation='sigmoid')
    ])
    
    model.compile(loss = 'binary_crossentropy', 
                  optimizer = 'adam', 
                  metrics = ['accuracy'])
    return model

# Define the early stopping callbacks
callbacks = tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            min_delta=0, 
            patience=10, verbose=2, 
            mode='auto', 
            restore_best_weights=True)

#==============================Step 4============================#
# Begin the training process with Average Word2Vec

# Parameter Initialization
oov_tok = "<UNK>"
columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 
           'acc7', 'acc8', 'acc9', 'acc10', 'AVG']

record_1 = pd.DataFrame(columns = columns)
record_2 = pd.DataFrame(columns = columns)
record_3 = pd.DataFrame(columns = columns)

sentences, labels = list(corpus.sentence), list(corpus.label)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

exp=0
acc_list_1 = []
acc_list_2 = []
acc_list_3 = []

# kfold.split() will return set indices for each split
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the data into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)
    
    # Define the word_index
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)
    word_index = tokenizer.word_index
    
    # Clean the sentences
    Xtrain = clean_doc(train_x, word_index)
    Xtest = clean_doc(test_x, word_index)

    # Encode the sentences into average Word2Vec representation
    Xtrain = encoded_sentences(Xtrain)
    Xtest = encoded_sentences(Xtest)
    
    # Define the input shape
    model_1 = define_model_1(Xtrain.shape[1])
    model_2 = define_model_2(Xtrain.shape[1])
    model_3 = define_model_3(Xtrain.shape[1])

    # Train the model
    model_1.fit(Xtrain, train_y, batch_size=32, 
                epochs=40, verbose=1, 
                callbacks=[callbacks], 
                validation_data=(Xtest, test_y))

    model_2.fit(Xtrain, train_y, batch_size=32, 
                epochs=40, verbose=1, 
                callbacks=[callbacks], 
                validation_data=(Xtest, test_y))
    
    model_3.fit(Xtrain, train_y, batch_size=32, 
                epochs=40, verbose=1, 
                callbacks=[callbacks], 
                validation_data=(Xtest, test_y))

    # evaluate the model
    loss_1, acc_1 = model_1.evaluate(Xtest, test_y, verbose=0)
    loss_2, acc_2 = model_2.evaluate(Xtest, test_y, verbose=0)
    loss_3, acc_3 = model_3.evaluate(Xtest, test_y, verbose=0)


    acc_list_1.append(acc_1)
    acc_list_2.append(acc_2)
    acc_list_3.append(acc_3)

mean_acc_1 = np.array(acc_list_1).mean()
mean_acc_2 = np.array(acc_list_2).mean()
mean_acc_3 = np.array(acc_list_3).mean()

entries_1 = acc_list_1 + [mean_acc_1]
entries_2 = acc_list_2 + [mean_acc_2]
entries_3 = acc_list_1 + [mean_acc_3]

temp = pd.DataFrame([entries_1], columns=columns)
record_1 = record_1.append(temp, ignore_index=True)
temp = pd.DataFrame([entries_2], columns=columns)
record_2 = record_2.append(temp, ignore_index=True)
temp = pd.DataFrame([entries_3], columns=columns)
record_3 = record_3.append(temp, ignore_index=True)

print(record_1)
print(record_2)
print(record_3)

#==============================Step 5============================#
# Save the dataframe into excel file
record_1.to_excel('WE_SNN_a.xlsx', sheet_name='model_1')
record_2.to_excel('WE_SNN_b.xlsx', sheet_name='model_2')
record_3.to_excel('WE_SNN_c.xlsx', sheet_name='model_3')
#================================================================#

# Python Program for CNN Models

In [ ]:
"""
Title            : Python Program for CNN Models
Dataset          : CR (as an example) 
Feature Extration: Random, Static, Dynamic Word2Vec
Author           : Diardano Raihan
"""

##=========================Import Libraries======================#
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from string import punctuation
from collections import Counter
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim.models import KeyedVectors
from sklearn.model_selection import KFold
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#==============================Step 1============================#
# Load the dataset
corpus = pd.read_pickle('../../0_data/CR/CR.pkl')
# Load the Word2Vec
word2vec = KeyedVectors.load_word2vec_format(
            '../GoogleNews-vectors-negative300.bin', 
            binary=True)

#==============================Step 2============================#
# Defined all the functions needed as Text Preprocessing steps

# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

# Calculated the statistics of Word2Vec
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()

# Map the Word2Vec into a matrix for embedding weights
def pretrained_embedding_matrix(word_to_vec_map, 
                                word_to_index, 
                                emb_mean, emb_std):
    
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution
    embed_matrix = np.random.normal(emb_mean, 
                                    emb_std, 
                                    (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th 
    # word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

#==============================Step 3============================#
# Model Definitions

def define_model(filters = 100, kernel_size = 3, 
                 activation='relu', input_dim = None, 
                 output_dim=300, max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, 
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
        tf.keras.layers.Conv1D(filters=filters, 
                               kernel_size = kernel_size, 
                               activation = activation, 
                               # set 'axis' value to the first and 
                               # second axis of conv1D weights 
                               # (rows, cols)
                               kernel_constraint = MaxNorm(
                                   max_value=3, 
                                   axis=[0,1])),
        
        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation=activation, 
                              # set axis to 0 to constrain 
                              # each weight vector of length 
                              # (input_dim,) in dense layer
                              kernel_constraint = MaxNorm(
                                  max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile(loss = 'binary_crossentropy', 
                  optimizer = 'adam', 
                  metrics = ['accuracy'])
    return model

def define_model_2(filters = 100, kernel_size = 3, 
                   activation='relu', input_dim = None, 
                   output_dim=300, max_length = None, 
                   emb_matrix = None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight 
                                  # with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not 
                                  # trainable (static)
                                  trainable = False),
        
        tf.keras.layers.Conv1D(filters=filters, 
                               kernel_size = kernel_size, 
                               activation = activation, 
                               # set 'axis' value to the first and 
                               # second axis of conv1D weights 
                               # (rows, cols)
                               kernel_constraint= MaxNorm(
                                   max_value=3, axis=[0,1])),
        
        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation=activation, 
                              # set axis to 0 to constrain 
                              # each weight vector of length 
                              # (input_dim,) in dense layer
                              kernel_constraint = MaxNorm(
                                  max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile(loss = 'binary_crossentropy', 
                  optimizer = 'adam', 
                  metrics = ['accuracy'])

    return model

def define_model_3(filters = 100, kernel_size = 3, 
                   activation='relu', input_dim = None, 
                   output_dim=300, max_length = None, 
                   emb_matrix = None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight 
                                  # with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not 
                                  # trainable (static)
                                  trainable = True),
        
        tf.keras.layers.Conv1D(filters=filters, 
                               kernel_size = kernel_size, 
                               activation = activation, 
                               # set 'axis' value to the first and 
                               # second axis of conv1D weights 
                               # (rows, cols)
                               kernel_constraint= MaxNorm(
                                   max_value=3, axis=[0,1])),
        
        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation=activation, 
                              # set axis to 0 to constrain 
                              # each weight vector of length 
                              # (input_dim,) in dense layer
                              kernel_constraint = MaxNorm(
                                  max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile(loss = 'binary_crossentropy', 
                  optimizer = 'adam', 
                  metrics = ['accuracy'])

    return model


# Define the early stopping callbacks
callbacks = tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            min_delta=0, 
            patience=10, verbose=2, 
            mode='auto', 
            restore_best_weights=True)

#==============================Step 3============================#
# Begin the training process with random/static/dynamic Word2Vec

# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu', 'tanh']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 
           'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 
           'acc10', 'AVG']

record_1 = pd.DataFrame(columns = columns)
record_2 = pd.DataFrame(columns = columns)
record_3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list_1 = []
        acc_list_2 = []
        acc_list_3 = []
        
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(
                                 train_x)
            test_sequences = tokenizer.texts_to_sequences(
                             test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, 
                                   maxlen=max_len, 
                                   padding=padding_type, 
                                   truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, 
                                  maxlen=max_len, 
                                  padding=padding_type, 
                                  truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, 
                                                     word_index, 
                                                     emb_mean, 
                                                     emb_std)
            
            # Define the models to train
            model_1 = define_model(filters, 
                                   kernel_size, 
                                   activation, 
                                   input_dim=vocab_size, 
                                   max_length=max_len)
            
            model_2 = define_model_2(filters, 
                                     kernel_size, 
                                     activation, 
                                     input_dim=vocab_size, 
                                     max_length=max_len, 
                                     emb_matrix=emb_matrix)
            
            model_3 = define_model_3(filters, 
                                     kernel_size, 
                                     activation, 
                                     input_dim=vocab_size, 
                                     max_length=max_len, 
                                     emb_matrix=emb_matrix)
            

            # Train the models
            model_1.fit(Xtrain, train_y, batch_size=50, 
                        epochs=100, verbose=1, 
                        callbacks=[callbacks], 
                        validation_data=(Xtest, test_y))

            model_2.fit(Xtrain, train_y, batch_size=50, 
                        epochs=100, verbose=1,
                        callbacks=[callbacks], 
                        validation_data=(Xtest, test_y))
            
            model_3.fit(Xtrain, train_y, batch_size=50, 
                        epochs=100, verbose=1, 
                      callbacks=[callbacks], 
                        validation_data=(Xtest, test_y))

            # evaluate the model
            loss_1, acc_1 = model_1.evaluate(Xtest, test_y)
            loss_2, acc_2 = model_2.evaluate(Xtest, test_y)
            loss_3, acc_3 = model_3.evaluate(Xtest, test_y)

            acc_list_1.append(acc*100)
            acc_list_2.append(acc*100)
            acc_list_3.append(acc*100)
            
        mean_acc_1 = np.array(acc_list_1).mean()
        mean_acc_2 = np.array(acc_list_2).mean()
        mean_acc_3 = np.array(acc_list_3).mean()
        
        parameters = [activation, kernel_size]
        entries_1 = parameters + acc_list + [mean_acc_1]
        entries_2 = parameters + acc_list + [mean_acc_2]
        entries_3 = parameters + acc_list + [mean_acc_3]
        
        temp = pd.DataFrame([entries_1], columns=columns)
        record_1 = record_1.append(temp, ignore_index=True)
        temp = pd.DataFrame([entries_2], columns=columns)
        record_2 = record_2.append(temp, ignore_index=True)
        temp = pd.DataFrame([entries_3], columns=columns)
        record_3 = record_3.append(temp, ignore_index=True)

        print(record_1)
        print(record_2)
        print(record_3)

#==============================Step 5============================#
# Save the dataframe into excel file
record_1.to_excel('WE_CNN_1.xlsx', sheet_name='random')
record_2.to_excel('WE_CNN_2.xlsx', sheet_name='static')
record_3.to_excel('WE_CNN_3.xlsx', sheet_name='dynamic')
#================================================================#

# Python Program for TCN Models

In [ ]:
"""
Title            : Python Program for TCN Models
Dataset          : CR (as an example) 
Feature Extration: Random, Static, Dynamic Word2Vec
Author           : Diardano Raihan (inspired by: christofhenkel)
https://www.kaggle.com/christofhenkel/temporal-convolutional-network
"""

##=========================Import Libraries======================#
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from string import punctuation
from collections import Counter
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim.models import KeyedVectors
from sklearn.model_selection import KFold
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tcn import TCN, tcn_full_summary
from tensorflow.keras.layers import Input, Embedding
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.models import Model

#==============================Step 1============================#
# Load the dataset
corpus = pd.read_pickle('../../0_data/CR/CR.pkl')
# Load the Word2Vec
word2vec = KeyedVectors.load_word2vec_format(
            '../GoogleNews-vectors-negative300.bin', 
            binary=True)

#==============================Step 2============================#
# Defined all the functions needed as Text Preprocessing steps

# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

# Calculated the statistics of Word2Vec
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()

# Map the Word2Vec into a matrix for embedding weights
def pretrained_embedding_matrix(word_to_vec_map, 
                                word_to_index, 
                                emb_mean, emb_std):
    
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution
    embed_matrix = np.random.normal(emb_mean, 
                                    emb_std, 
                                    (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th 
    # word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

#==============================Step 3============================#
# TCN-rand
def define_model(kernel_size = 3, activation='relu', 
                 input_dim = None, output_dim=300, 
                 max_length = None ):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, 
                  output_dim=output_dim, 
                  input_length=max_length)(inp)
    
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4],
            kernel_size = kernel_size,
            return_sequences=True, 
            activation = activation, 
            name = 'tcn1')(x)
    
    x = TCN(64,dilations = [1, 2, 4], 
            kernel_size = kernel_size,
            return_sequences=True, 
            activation = activation, 
            name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss = 'binary_crossentropy', 
                  optimizer = 'adam', 
                  metrics = ['accuracy'])
    
    return model

# TCN-static
def define_model_2(kernel_size = 3, activation='relu', 
                   input_dim = None, 
                   output_dim=300, max_length = None, 
                   emb_matrix = None):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, 
                  output_dim=output_dim, 
                  input_length=max_length,
                  # Assign the embedding weight with 
                  # word2vec embedding marix
                  weights = [emb_matrix],
                  # Set the weight to be not trainable 
                  # (static)
                  trainable = False)(inp)
    
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4],
            kernel_size = kernel_size,
            return_sequences=True, 
            activation = activation, 
            name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4],
            kernel_size = kernel_size,
            return_sequences=True, 
            activation = activation, 
            name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'binary_crossentropy', 
                  optimizer = 'adam', 
                  metrics = ['accuracy'])
    
    return model

# TCN-dynamic
def define_model_3(kernel_size = 3, activation='relu', 
                   input_dim = None, 
                   output_dim=300, max_length = None, 
                   emb_matrix = None):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, 
                  output_dim=output_dim, 
                  input_length=max_length,
                  # Assign the embedding weight with 
                  # word2vec embedding marix
                  weights = [emb_matrix],
                  # Set the weight to be not trainable 
                  # (static)
                  trainable = True)(inp)
    
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4],
            kernel_size = kernel_size,
            return_sequences=True, 
            activation = activation, 
            name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4],
            kernel_size = kernel_size,
            return_sequences=True, 
            activation = activation, 
            name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'binary_crossentropy', 
                  optimizer = 'adam', 
                  metrics = ['accuracy'])
    
    return model

# Define the early stopping callbacks
callbacks = tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            min_delta=0, 
            patience=10, verbose=2, 
            mode='auto', 
            restore_best_weights=True)

#==============================Step 3============================#
# Begin the training process with random/static/dynamic Word2Vec

# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu', 'tanh']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 
           'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 
           'acc10', 'AVG']

record_1 = pd.DataFrame(columns = columns)
record_2 = pd.DataFrame(columns = columns)
record_3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list_1 = []
        acc_list_2 = []
        acc_list_3 = []
        
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(
                                 train_x)
            test_sequences = tokenizer.texts_to_sequences(
                             test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, 
                                   maxlen=max_len, 
                                   padding=padding_type, 
                                   truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, 
                                  maxlen=max_len, 
                                  padding=padding_type, 
                                  truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, 
                                                     word_index, 
                                                     emb_mean, 
                                                     emb_std)
            
            # Define the models to train
            model_1 = define_model(filters, 
                                   kernel_size, 
                                   activation, 
                                   input_dim=vocab_size, 
                                   max_length=max_len)
            
            model_2 = define_model_2(filters, 
                                     kernel_size, 
                                     activation, 
                                     input_dim=vocab_size, 
                                     max_length=max_len, 
                                     emb_matrix=emb_matrix)
            
            model_3 = define_model_3(filters, 
                                     kernel_size, 
                                     activation, 
                                     input_dim=vocab_size, 
                                     max_length=max_len, 
                                     emb_matrix=emb_matrix)
            

            # Train the models
            model_1.fit(Xtrain, train_y, batch_size=50, 
                        epochs=100, verbose=1, 
                        callbacks=[callbacks], 
                        validation_data=(Xtest, test_y))

            model_2.fit(Xtrain, train_y, batch_size=50, 
                        epochs=100, verbose=1,
                        callbacks=[callbacks], 
                        validation_data=(Xtest, test_y))
            
            model_3.fit(Xtrain, train_y, batch_size=50, 
                        epochs=100, verbose=1, 
                      callbacks=[callbacks], 
                        validation_data=(Xtest, test_y))

            # evaluate the model
            loss_1, acc_1 = model_1.evaluate(Xtest, test_y)
            loss_2, acc_2 = model_2.evaluate(Xtest, test_y)
            loss_3, acc_3 = model_3.evaluate(Xtest, test_y)

            acc_list_1.append(acc*100)
            acc_list_2.append(acc*100)
            acc_list_3.append(acc*100)
            
        mean_acc_1 = np.array(acc_list_1).mean()
        mean_acc_2 = np.array(acc_list_2).mean()
        mean_acc_3 = np.array(acc_list_3).mean()
        
        parameters = [activation, kernel_size]
        entries_1 = parameters + acc_list + [mean_acc_1]
        entries_2 = parameters + acc_list + [mean_acc_2]
        entries_3 = parameters + acc_list + [mean_acc_3]
        
        temp = pd.DataFrame([entries_1], columns=columns)
        record_1 = record_1.append(temp, ignore_index=True)
        temp = pd.DataFrame([entries_2], columns=columns)
        record_2 = record_2.append(temp, ignore_index=True)
        temp = pd.DataFrame([entries_3], columns=columns)
        record_3 = record_3.append(temp, ignore_index=True)

        print(record_1)
        print(record_2)
        print(record_3)

#==============================Step 5============================#
# Save the dataframe into excel file
record_1.to_excel('WE_CNN_1.xlsx', sheet_name='random')
record_2.to_excel('WE_CNN_2.xlsx', sheet_name='static')
record_3.to_excel('WE_CNN_3.xlsx', sheet_name='dynamic')
#================================================================#

# Python Program for BiGRU/LSTM Models

In [ ]:
"""
Title            : Python Program for BiGRU/LSTM Models
Dataset          : CR (as an example)
Feature Extration: Random, Static, Dynamic Word2Vec
Author           : Diardano Raihan
"""

##=========================Import Libraries======================#
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from string import punctuation
from collections import Counter
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim.models import KeyedVectors
from sklearn.model_selection import KFold
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#==============================Step 1============================#
# Load the dataset
corpus = pd.read_pickle('../../0_data/CR/CR.pkl')
# Load the Word2Vec
word2vec = KeyedVectors.load_word2vec_format(
            '../GoogleNews-vectors-negative300.bin', 
            binary=True)

#==============================Step 2============================#
# Defined all the functions needed as Text Preprocessing steps

# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

# Calculated the statistics of Word2Vec
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()

# Map the Word2Vec into a matrix for embedding weights
def pretrained_embedding_matrix(word_to_vec_map, 
                                word_to_index, 
                                emb_mean, emb_std):
    
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution
    embed_matrix = np.random.normal(emb_mean, 
                                    emb_std, 
                                    (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th 
    # word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

#==============================Step 3============================#
# Model Definitions

from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(input_dim = None, 
                 output_dim=300, 
                 max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
        # Change it to tf.keras.layers.LSTM(64) for LSTM version
        # inside the Bidirectional layer
        tf.keras.layers.Bidirectional((tf.keras.layers.GRU(64))),
        tf.keras.layers.Dropout(0.5),
        # Propagate X through a Dense layer with 1 unit
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile(loss = 'binary_crossentropy', 
                  optimizer = 'adam', 
                  metrics = ['accuracy'])

    return model

from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_2(input_dim = None, 
                   output_dim=300, 
                   max_length = None, 
                   emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight 
                                  # with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not 
                                  # trainable (static)
                                  trainable = False),
        
        # Change it to tf.keras.layers.LSTM(64) for LSTM version
        # inside the Bidirectional layer        
        tf.keras.layers.Bidirectional((tf.keras.layers.GRU(64))),
        tf.keras.layers.Dropout(0.5),
        # Propagate X through a Dense layer with 1 unit
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile(loss = 'binary_crossentropy', 
                  optimizer = 'adam', 
                  metrics = ['accuracy'])

    return model

def define_model_3(input_dim = None, 
                   output_dim=300, 
                   max_length = None, 
                   emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight 
                                  # with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not 
                                  # trainable (static)
                                  trainable = True),
        
        # Change it to tf.keras.layers.LSTM(64) for LSTM version
        # inside the Bidirectional layer        
        tf.keras.layers.Bidirectional((tf.keras.layers.GRU(64))),
        tf.keras.layers.Dropout(0.5),
        # Propagate X through a Dense layer with 1 unit
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile(loss = 'binary_crossentropy', 
                  optimizer = 'adam', 
                  metrics = ['accuracy'])

    return model

# Define the early stopping callbacks
callbacks = tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            min_delta=0, 
            patience=10, verbose=2, 
            mode='auto', 
            restore_best_weights=True)

#==============================Step 3============================#
# Begin the training process with random/static/dynamic Word2Vec

# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu', 'tanh']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 
           'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 
           'acc10', 'AVG']

record_1 = pd.DataFrame(columns = columns)
record_2 = pd.DataFrame(columns = columns)
record_3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list_1 = []
        acc_list_2 = []
        acc_list_3 = []
        
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(
                                 train_x)
            test_sequences = tokenizer.texts_to_sequences(
                             test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, 
                                   maxlen=max_len, 
                                   padding=padding_type, 
                                   truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, 
                                  maxlen=max_len, 
                                  padding=padding_type, 
                                  truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, 
                                                     word_index, 
                                                     emb_mean, 
                                                     emb_std)
            
            # Define the models to train
            model_1 = define_model(filters, 
                                   kernel_size, 
                                   activation, 
                                   input_dim=vocab_size, 
                                   max_length=max_len)
            
            model_2 = define_model_2(filters, 
                                     kernel_size, 
                                     activation, 
                                     input_dim=vocab_size, 
                                     max_length=max_len, 
                                     emb_matrix=emb_matrix)
            
            model_3 = define_model_3(filters, 
                                     kernel_size, 
                                     activation, 
                                     input_dim=vocab_size, 
                                     max_length=max_len, 
                                     emb_matrix=emb_matrix)
            

            # Train the models
            model_1.fit(Xtrain, train_y, batch_size=50, 
                        epochs=100, verbose=1, 
                        callbacks=[callbacks], 
                        validation_data=(Xtest, test_y))

            model_2.fit(Xtrain, train_y, batch_size=50, 
                        epochs=100, verbose=1,
                        callbacks=[callbacks], 
                        validation_data=(Xtest, test_y))
            
            model_3.fit(Xtrain, train_y, batch_size=50, 
                        epochs=100, verbose=1, 
                      callbacks=[callbacks], 
                        validation_data=(Xtest, test_y))

            # evaluate the model
            loss_1, acc_1 = model_1.evaluate(Xtest, test_y)
            loss_2, acc_2 = model_2.evaluate(Xtest, test_y)
            loss_3, acc_3 = model_3.evaluate(Xtest, test_y)

            acc_list_1.append(acc*100)
            acc_list_2.append(acc*100)
            acc_list_3.append(acc*100)
            
        mean_acc_1 = np.array(acc_list_1).mean()
        mean_acc_2 = np.array(acc_list_2).mean()
        mean_acc_3 = np.array(acc_list_3).mean()
        
        parameters = [activation, kernel_size]
        entries_1 = parameters + acc_list + [mean_acc_1]
        entries_2 = parameters + acc_list + [mean_acc_2]
        entries_3 = parameters + acc_list + [mean_acc_3]
        
        temp = pd.DataFrame([entries_1], columns=columns)
        record_1 = record_1.append(temp, ignore_index=True)
        temp = pd.DataFrame([entries_2], columns=columns)
        record_2 = record_2.append(temp, ignore_index=True)
        temp = pd.DataFrame([entries_3], columns=columns)
        record_3 = record_3.append(temp, ignore_index=True)

        print(record_1)
        print(record_2)
        print(record_3)

#==============================Step 5============================#
# Save the dataframe into excel file
record_1.to_excel('WE_CNN_1.xlsx', sheet_name='random')
record_2.to_excel('WE_CNN_2.xlsx', sheet_name='static')
record_3.to_excel('WE_CNN_3.xlsx', sheet_name='dynamic')
#================================================================#

# Python Program for Stacked BiGRU/LSTM Models

In [ ]:
"""
Title            : Python Program for Stacked BiGRU/LSTM Models
Dataset          : CR (as an example)
Feature Extration: Random, Static, Dynamic Word2Vec
Author           : Diardano Raihan
"""

##=========================Import Libraries======================#
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from string import punctuation
from collections import Counter
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim.models import KeyedVectors
from sklearn.model_selection import KFold
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#==============================Step 1============================#
# Load the dataset
corpus = pd.read_pickle('../../0_data/CR/CR.pkl')
# Load the Word2Vec
word2vec = KeyedVectors.load_word2vec_format(
            '../GoogleNews-vectors-negative300.bin', 
            binary=True)

#==============================Step 2============================#
# Defined all the functions needed as Text Preprocessing steps

# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

# Calculated the statistics of Word2Vec
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()

# Map the Word2Vec into a matrix for embedding weights
def pretrained_embedding_matrix(word_to_vec_map, 
                                word_to_index, 
                                emb_mean, emb_std):
    
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution
    embed_matrix = np.random.normal(emb_mean, 
                                    emb_std, 
                                    (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th 
    # word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

#==============================Step 3============================#
# Model Definitions

from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(input_dim = None, 
                 output_dim=300, 
                 max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
        # Change it to tf.keras.layers.LSTM(64) for LSTM version
        # inside the Bidirectional layer
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
                                      64, 
                                      return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
                                      64, 
                                      return_sequences=False)),
        tf.keras.layers.Dropout(0.5),
        # Propagate X through a Dense layer with 1 unit
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile(loss = 'binary_crossentropy', 
                  optimizer = 'adam', 
                  metrics = ['accuracy'])

    return model

from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_2(input_dim = None, 
                   output_dim=300, 
                   max_length = None, 
                   emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight 
                                  # with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not 
                                  # trainable (static)
                                  trainable = False),
        
        # Change it to tf.keras.layers.LSTM(64) for LSTM version
        # inside the Bidirectional layer
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
                                      64, 
                                      return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
                                      64, 
                                      return_sequences=False)),
        tf.keras.layers.Dropout(0.5),
        # Propagate X through a Dense layer with 1 unit
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile(loss = 'binary_crossentropy', 
                  optimizer = 'adam', 
                  metrics = ['accuracy'])

    return model

def define_model_3(input_dim = None, 
                   output_dim=300, 
                   max_length = None, 
                   emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight 
                                  # with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not 
                                  # trainable (static)
                                  trainable = True),
        
        # Change it to tf.keras.layers.LSTM(64) for LSTM version
        # inside the Bidirectional layer
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
                                      64, 
                                      return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
                                      64, 
                                      return_sequences=False)),
        tf.keras.layers.Dropout(0.5),
        # Propagate X through a Dense layer with 1 unit
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile(loss = 'binary_crossentropy', 
                  optimizer = 'adam', 
                  metrics = ['accuracy'])

    return model

# Define the early stopping callbacks
callbacks = tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            min_delta=0, 
            patience=10, verbose=2, 
            mode='auto', 
            restore_best_weights=True)

#==============================Step 3============================#
# Begin the training process with random/static/dynamic Word2Vec

# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu', 'tanh']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 
           'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 
           'acc10', 'AVG']

record_1 = pd.DataFrame(columns = columns)
record_2 = pd.DataFrame(columns = columns)
record_3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list_1 = []
        acc_list_2 = []
        acc_list_3 = []
        
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(
                                 train_x)
            test_sequences = tokenizer.texts_to_sequences(
                             test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, 
                                   maxlen=max_len, 
                                   padding=padding_type, 
                                   truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, 
                                  maxlen=max_len, 
                                  padding=padding_type, 
                                  truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, 
                                                     word_index, 
                                                     emb_mean, 
                                                     emb_std)
            
            # Define the models to train
            model_1 = define_model(filters, 
                                   kernel_size, 
                                   activation, 
                                   input_dim=vocab_size, 
                                   max_length=max_len)
            
            model_2 = define_model_2(filters, 
                                     kernel_size, 
                                     activation, 
                                     input_dim=vocab_size, 
                                     max_length=max_len, 
                                     emb_matrix=emb_matrix)
            
            model_3 = define_model_3(filters, 
                                     kernel_size, 
                                     activation, 
                                     input_dim=vocab_size, 
                                     max_length=max_len, 
                                     emb_matrix=emb_matrix)
            

            # Train the models
            model_1.fit(Xtrain, train_y, batch_size=50, 
                        epochs=100, verbose=1, 
                        callbacks=[callbacks], 
                        validation_data=(Xtest, test_y))

            model_2.fit(Xtrain, train_y, batch_size=50, 
                        epochs=100, verbose=1,
                        callbacks=[callbacks], 
                        validation_data=(Xtest, test_y))
            
            model_3.fit(Xtrain, train_y, batch_size=50, 
                        epochs=100, verbose=1, 
                      callbacks=[callbacks], 
                        validation_data=(Xtest, test_y))

            # evaluate the model
            loss_1, acc_1 = model_1.evaluate(Xtest, test_y)
            loss_2, acc_2 = model_2.evaluate(Xtest, test_y)
            loss_3, acc_3 = model_3.evaluate(Xtest, test_y)

            acc_list_1.append(acc*100)
            acc_list_2.append(acc*100)
            acc_list_3.append(acc*100)
            
        mean_acc_1 = np.array(acc_list_1).mean()
        mean_acc_2 = np.array(acc_list_2).mean()
        mean_acc_3 = np.array(acc_list_3).mean()
        
        parameters = [activation, kernel_size]
        entries_1 = parameters + acc_list + [mean_acc_1]
        entries_2 = parameters + acc_list + [mean_acc_2]
        entries_3 = parameters + acc_list + [mean_acc_3]
        
        temp = pd.DataFrame([entries_1], columns=columns)
        record_1 = record_1.append(temp, ignore_index=True)
        temp = pd.DataFrame([entries_2], columns=columns)
        record_2 = record_2.append(temp, ignore_index=True)
        temp = pd.DataFrame([entries_3], columns=columns)
        record_3 = record_3.append(temp, ignore_index=True)

        print(record_1)
        print(record_2)
        print(record_3)

#==============================Step 5============================#
# Save the dataframe into excel file
record_1.to_excel('WE_StackedBiGRU/LSTM_1.xlsx', sheet_name='random')
record_2.to_excel('WE_StackedBiGRU/LSTM_2.xlsx', sheet_name='static')
record_3.to_excel('WE_StackedBiGRU/LSTM_3.xlsx', sheet_name='dynamic')
#================================================================#

# Python Program for Ensemble Learning-based Models

In [ ]:
"""
Title            : Python Program for Ensemble-based Models
Dataset          : CR (as an example)
Feature Extration: Random, Static, Dynamic Word2Vec
Author           : Diardano Raihan
"""

##=========================Import Libraries======================#
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from string import punctuation
from collections import Counter
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim.models import KeyedVectors
from sklearn.model_selection import KFold
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Input, Embedding, Conv1D
from tensorflow.keras.layers import Dropout, MaxPool1D, Flatten
from tensorflow.keras.layers import Dense, Bidirectional, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate

#==============================Step 1============================#
# Load the dataset
corpus = pd.read_pickle('../../0_data/CR/CR.pkl')
# Load the Word2Vec
word2vec = KeyedVectors.load_word2vec_format(
            '../GoogleNews-vectors-negative300.bin', 
            binary=True)

#==============================Step 2============================#
# Defined all the functions needed as Text Preprocessing steps

# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

# Calculated the statistics of Word2Vec
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()

# Map the Word2Vec into a matrix for embedding weights
def pretrained_embedding_matrix(word_to_vec_map, 
                                word_to_index, 
                                emb_mean, emb_std):
    
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution
    embed_matrix = np.random.normal(emb_mean, 
                                    emb_std, 
                                    (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th 
    # word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

#==============================Step 3============================#
# Model Definitions

def define_model(filters = 100, kernel_size = 3, 
                 activation='relu', input_dim = None, 
                 output_dim=300, max_length = None ):
  
    # Channel 1
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length)(input1)
    conv1 = Conv1D(filters=filters, 
                   kernel_size=kernel_size, 
                   activation='relu', 
                   kernel_constraint= MaxNorm(max_value=3, 
                                              axis=[0,1])
                  )(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(1, activation='sigmoid')(drop1)
    
    # Channel 2
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            mask_zero=True)(input2)
    
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(1, activation='sigmoid')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(1, activation='sigmoid')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model.compile( loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    
    return model

def define_model_2(filters = 100, kernel_size = 3, 
                   activation='relu', 
                   input_dim = None, output_dim=300, 
                   max_length = None, emb_matrix = None):
  
    # Channel 1
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight 
                            # with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable 
                            # (static)
                            trainable = False)(input1)
    conv1 = Conv1D(filters=filters, 
                   kernel_size=kernel_size, 
                   activation='relu', 
                   kernel_constraint= MaxNorm(max_value=3, 
                                              axis=[0,1])
                  )(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(1, activation='sigmoid')(drop1)
    
    # Channel 2
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight 
                            # with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable 
                            # (static)
                            trainable = False,
                            mask_zero=True)(input2)
    
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(1, activation='sigmoid')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(1, activation='sigmoid')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model.compile( loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    
    return model

def define_model_3(filters = 100, kernel_size = 3, 
                   activation='relu', 
                   input_dim = None, output_dim=300, 
                   max_length = None, emb_matrix = None):
  
    # Channel 1
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight 
                            # with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable 
                            # (static)
                            trainable = True)(input1)
    conv1 = Conv1D(filters=filters, 
                   kernel_size=kernel_size, 
                   activation='relu', 
                   kernel_constraint= MaxNorm(max_value=3, 
                                              axis=[0,1])
                  )(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(1, activation='sigmoid')(drop1)
    
    # Channel 2
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight 
                            # with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable 
                            # (static)
                            trainable = True,
                            mask_zero=True)(input2)
    
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(1, activation='sigmoid')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(1, activation='sigmoid')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model.compile( loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    
    return model

# Define the early stopping callbacks
callbacks = tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            min_delta=0, 
            patience=10, verbose=2, 
            mode='auto', 
            restore_best_weights=True)

#==============================Step 3============================#
# Begin the training process with random/static/dynamic Word2Vec

# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu', 'tanh']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 
           'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 
           'acc10', 'AVG']

record_1 = pd.DataFrame(columns = columns)
record_2 = pd.DataFrame(columns = columns)
record_3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list_1 = []
        acc_list_2 = []
        acc_list_3 = []
        
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(
                                 train_x)
            test_sequences = tokenizer.texts_to_sequences(
                             test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, 
                                   maxlen=max_len, 
                                   padding=padding_type, 
                                   truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, 
                                  maxlen=max_len, 
                                  padding=padding_type, 
                                  truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, 
                                                     word_index, 
                                                     emb_mean, 
                                                     emb_std)
            
            # Define the models to train
            model_1 = define_model(filters, 
                                   kernel_size, 
                                   activation, 
                                   input_dim=vocab_size, 
                                   max_length=max_len)
            
            model_2 = define_model_2(filters, 
                                     kernel_size, 
                                     activation, 
                                     input_dim=vocab_size, 
                                     max_length=max_len, 
                                     emb_matrix=emb_matrix)
            
            model_3 = define_model_3(filters, 
                                     kernel_size, 
                                     activation, 
                                     input_dim=vocab_size, 
                                     max_length=max_len, 
                                     emb_matrix=emb_matrix)
            

            # Train the models
            model_1.fit(x=[Xtrain, Xtrain], train_y, batch_size=50, 
                        epochs=100, verbose=1, 
                        callbacks=[callbacks], 
                        validation_data=([Xtest, Xtest], test_y))

            model_2.fit(x=[Xtrain, Xtrain], train_y, batch_size=50, 
                        epochs=100, verbose=1,
                        callbacks=[callbacks], 
                        validation_data=([Xtest, Xtest], test_y))
            
            model_3.fit(x=[Xtrain, Xtrain], train_y, batch_size=50, 
                        epochs=100, verbose=1, 
                      callbacks=[callbacks], 
                        validation_data=([Xtest, Xtest], test_y))

            # evaluate the model
            loss_1, acc_1 = model_1.evaluate([Xtest, Xtest], test_y)
            loss_2, acc_2 = model_2.evaluate([Xtest, Xtest], test_y)
            loss_3, acc_3 = model_3.evaluate([Xtest, Xtest], test_y)

            acc_list_1.append(acc*100)
            acc_list_2.append(acc*100)
            acc_list_3.append(acc*100)
            
        mean_acc_1 = np.array(acc_list_1).mean()
        mean_acc_2 = np.array(acc_list_2).mean()
        mean_acc_3 = np.array(acc_list_3).mean()
        
        parameters = [activation, kernel_size]
        entries_1 = parameters + acc_list + [mean_acc_1]
        entries_2 = parameters + acc_list + [mean_acc_2]
        entries_3 = parameters + acc_list + [mean_acc_3]
        
        temp = pd.DataFrame([entries_1], columns=columns)
        record_1 = record_1.append(temp, ignore_index=True)
        temp = pd.DataFrame([entries_2], columns=columns)
        record_2 = record_2.append(temp, ignore_index=True)
        temp = pd.DataFrame([entries_3], columns=columns)
        record_3 = record_3.append(temp, ignore_index=True)

        print(record_1)
        print(record_2)
        print(record_3)

#==============================Step 5============================#
# Save the dataframe into excel file
record_1.to_excel('WE_Ensemble_1.xlsx', sheet_name='random')
record_2.to_excel('WE_Ensemble_2.xlsx', sheet_name='static')
record_3.to_excel('WE_Ensemble_3.xlsx', sheet_name='dynamic')
#================================================================#